# Project 1 - Adult and Wine Dataset

In [ ]:
Names = "Zafar Khan. Subrat Acharya, and Michael Dobroski"

In [2]:
# imports
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import seaborn as sns
from sklearn import (datasets, neighbors,
                     naive_bayes,
                     model_selection as skms,
                     linear_model, dummy,
                     metrics, preprocessing)
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

# Adult Dataset

In [3]:
######### Beginning of Adult dataset

# Reading in Files, made a slight typo in naming my Adulttest.csv file
adult_train_plus_val_df = pd.read_csv("Adulttrain.csv")
adult_test_df = pd.read_csv("Aduttest.csv")

#Replacing ? with the most common feature in each coloumn
adult_train_plus_val_df['workclass'] = adult_train_plus_val_df['workclass'].replace({'?':"Private"})
adult_train_plus_val_df['occupation'] = adult_train_plus_val_df['occupation'].replace({'?':"Other-service"})
adult_train_plus_val_df['native-country'] = adult_train_plus_val_df['native-country'].replace({'?':"United-States"})

#Doing the same thing above but with the test dataframe
adult_test_df['workclass'] = adult_test_df['workclass'].replace({'?':"Private"})
adult_test_df['occupation'] = adult_test_df['occupation'].replace({'?':"Other-service"})
adult_test_df['native-country'] = adult_test_df['native-country'].replace({'?':"United-States"})

# Source: https://www.youtube.com/watch?v=9yl6-HEY7_s (used below)
# These dummy dataframes are essentially one hot encoded data frames
workclass = pd.get_dummies(adult_train_plus_val_df['workclass'])
marital_status = pd.get_dummies(adult_train_plus_val_df['marital-status'])
occupation = pd.get_dummies(adult_train_plus_val_df['occupation'])
relationship =pd.get_dummies(adult_train_plus_val_df['relationship'])
race = pd.get_dummies(adult_train_plus_val_df['race'])
sex =pd.get_dummies(adult_train_plus_val_df['sex'])
native_country = pd.get_dummies(adult_train_plus_val_df['native-country'])


workclass2 = pd.get_dummies(adult_test_df['workclass'])
marital_status2 = pd.get_dummies(adult_test_df['marital-status'])
occupation2 = pd.get_dummies(adult_test_df['occupation'])
relationship2 =pd.get_dummies(adult_test_df['relationship'])
race2 = pd.get_dummies(adult_test_df['race'])
sex2 =pd.get_dummies(adult_test_df['sex'])
native_country2 = pd.get_dummies(adult_test_df['native-country'])
#

#Dropping all the columns with strings from both dataframes
adult_train_plus_val_df = adult_train_plus_val_df.drop(['workclass'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['marital-status'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['occupation'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['relationship'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['race'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['sex'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['native-country'],axis=1)

#Same thing as above
adult_test_df = adult_test_df.drop(['workclass'],axis=1)
adult_test_df = adult_test_df.drop(['marital-status'],axis=1)
adult_test_df = adult_test_df.drop(['occupation'],axis=1)
adult_test_df = adult_test_df.drop(['relationship'],axis=1)
adult_test_df = adult_test_df.drop(['race'],axis=1)
adult_test_df = adult_test_df.drop(['sex'],axis=1)
adult_test_df = adult_test_df.drop(['native-country'],axis=1)

# Dropping columns which we found unneccesary
adult_train_plus_val_df = adult_train_plus_val_df.drop(['fnlwgt'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['education'],axis=1)
adult_train_plus_val_df = adult_train_plus_val_df.drop(['id'],axis=1)

# Same thing as above however, had to keep id for predicting in test dataframe
adult_test_df = adult_test_df.drop(['fnlwgt'],axis=1)
adult_test_df = adult_test_df.drop(['education'],axis=1)
adult_test_df_id = adult_test_df.copy()
adult_test_df = adult_test_df.drop(['id'],axis=1)

#All the one hot encoded dataframes excluding a few which we found unnecessary (Listed beneath)
frames = [workclass, marital_status, occupation, sex, adult_train_plus_val_df]
frames2 = [workclass2, marital_status2, occupation2, sex2, adult_test_df]
# Relationships, Native Country, Race

# Finally done preprocessing data by adding all dataframes together in repspective original dataframes
adult_train_plus_val_df = pd.concat(frames, axis =1)
adult_test_df = pd.concat(frames2, axis =1)

# Dropping target class
adult_train_plus_val_tgt = adult_train_plus_val_df["class"]
adult_train_plus_val_ftrs = adult_train_plus_val_df.drop(['class'],axis=1)

# Here we attempted to scale our data but we discovered it to be a hinderance to our score, eventually got rid of it
#min_max = preprocessing.MinMaxScaler(feature_range = (0,1))
#for x in ['education-num', 'capital-gain', 'capital-loss', 'age', 'hours-per-week'] :
#    adult_train_plus_val_ftrs[[x]] = min_max.fit_transform(adult_train_plus_val_ftrs[[x]])
    
#for x in ['education-num', 'capital-gain', 'capital-loss', 'age', 'hours-per-week'] :
#    adult_test_df[[x]] = min_max.fit_transform(adult_test_df[[x]])

# Train test split our train and validation data
(adult_train_ftrs,
 adult_val_ftrs,
 adult_train_tgt,
 adult_val_tgt) = skms.train_test_split(adult_train_plus_val_ftrs,
                                                      adult_train_plus_val_tgt,
                                                      test_size=.25)



In [4]:
# add k-NN models with various values of k to models_to_try
models_to_try = {'NB': naive_bayes.GaussianNB()}
for k in range(1,32,2):
    models_to_try[f'{k}-NN'] = neighbors.KNeighborsClassifier(n_neighbors=k)

#Utilizing cross fold validation to find best model, utilized 5 folds in order to be somewhat effective
model_acc = {}
for model_name in models_to_try:
    scores = skms.cross_val_score(models_to_try[model_name],
                                  adult_train_plus_val_ftrs,
                                  adult_train_plus_val_tgt,
                                  cv=5,
                                  scoring='accuracy')
    accuracy = scores.mean()
    print(f'{model_name}: {accuracy:.3f}')
    model_acc[model_name] = accuracy


NB: 0.831
1-NN: 0.816
3-NN: 0.839
5-NN: 0.847
7-NN: 0.849
9-NN: 0.849
11-NN: 0.848
13-NN: 0.849
15-NN: 0.850
17-NN: 0.850
19-NN: 0.851
21-NN: 0.850
23-NN: 0.850
25-NN: 0.850
27-NN: 0.851
29-NN: 0.849
31-NN: 0.848


In [5]:
#Select best model from statements above to use as final model
best_model = neighbors.KNeighborsClassifier(n_neighbors=19)
fit = best_model.fit(adult_train_plus_val_ftrs, adult_train_plus_val_tgt)

In [6]:
# Make our predictions using the model we chose above and lastly writing out to a file
predictions = best_model.predict(adult_test_df)
predictions_final = pd.DataFrame()
predictions_final['Id'] = adult_test_df_id['id']
predictions_final['Category'] = predictions
#predictions_final.to_csv("adultpredictionsK=19.csv", index=None) #Write out to new CSV file

########## End of Adult dataset

# Wine Data Set

In [7]:
########## Beginning of Wine dataset

wine_df = pd.read_csv('wine_train.csv')
wine_test = pd.read_csv('wine_test.csv')

wine_train_val_ftrs = wine_df.drop(['quality', 'id', 'density'], axis = 1)
wine_train_val_tgt = wine_df['quality']
wine_test_ftrs = wine_test.drop(['id', 'density'], axis = 1)

min_max = preprocessing.MinMaxScaler(feature_range = (0,1))
for col in wine_train_val_ftrs.columns:
    wine_train_val_ftrs[[col]] = min_max.fit_transform(wine_train_val_ftrs[[col]])
for col in wine_test_ftrs.columns:
    wine_test_ftrs[[col]] = min_max.fit_transform(wine_test_ftrs[[col]])

In [8]:
models_to_try = {'lr': linear_model.LinearRegression(), 
                 '1-NN': neighbors.KNeighborsRegressor(n_neighbors=1),
                 '3-NN': neighbors.KNeighborsRegressor(n_neighbors=3),
                 '5-NN': neighbors.KNeighborsRegressor(n_neighbors=5),
                 '6-NN': neighbors.KNeighborsRegressor(n_neighbors=6),
                 '7-NN': neighbors.KNeighborsRegressor(n_neighbors=7),
                 '9-NN': neighbors.KNeighborsRegressor(n_neighbors=9),}
model_rmse = {}
for model_name in models_to_try:
    scores = skms.cross_val_score(models_to_try[model_name],
                                  wine_train_val_ftrs,
                                  wine_train_val_tgt,
                                  cv=10,
                                  scoring='neg_mean_squared_error')
    
    mean_rmse = np.sqrt(-scores.mean())
    print(f'{model_name}: {mean_rmse:.2f}')
    model_rmse[model_name] = mean_rmse

best_model_name = min(model_rmse,key=model_rmse.get)
print(f'\nBest model: {best_model_name}; RMSE: {model_rmse[best_model_name]:.2f}')
best_model = models_to_try[best_model_name]

lr: 0.78
1-NN: 1.00
3-NN: 0.85
5-NN: 0.82
6-NN: 0.81
7-NN: 0.80
9-NN: 0.79

Best model: lr; RMSE: 0.78


In [9]:
#best_model = neighbors.KNeighborsRegressor(n_neighbors = 15)
best_fit = best_model.fit(wine_train_val_ftrs, wine_train_val_tgt)
predictions = best_fit.predict(wine_test_ftrs)
predictions_final = pd.DataFrame()
predictions_final['Id'] = wine_test['id']
predictions_final['Predicted'] = np.around(predictions, decimals = 0)
#display(predictions_final)
#predictions_final.to_csv("winepredictions.csv", index=None)